In [1]:
#Impotar la libreria de spark
from pyspark.sql import SparkSession


In [2]:
#Iniciar contexto de Spark 
spark = SparkSession.builder.appName("Analisis de Datos-Big data") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()


In [3]:
#Leer el dataset 
#Como nuestro dataset no tiene encabezado, establecemos header en false
#Nuestro dataframe utiliza los encabezados por default que es _c0, _c01,....., _cn
df = spark.read.load("colocar_ruta_del_dataset",
            format="csv", sep=",", inferSchema="true", header="false")

In [4]:
#Establecemos el nombre de las columnas de nuestro dataset
newNameColumns = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell","su_attempted", "num_root", "num_file_creations","num_shells", "num_access_files", "num_outbound_cmds","is_host_login", "is_guest_login", "count", "srv_count","serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate","label"]
df_rename = df.toDF(*newNameColumns)


In [5]:
#Se van contar la cantidad de datos distintos de la ultima columna que referencia
# a los diferentes tipos de ataques que se tiene
df_rename.groupBy("label").count().orderBy("count", ascending=False).show()

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
+----------------+-------+
only showing top 20 rows



In [6]:
#Como K-mean trabaja solamente con valores numericos eliminarios algunas columnas no numericas
#Columna 1 -> Tipo de conexion
#Columna 2 -> Tipo de servicio
#Columna 3 -> Si el usuario inicio sesión o no
df_only_numeric = df_rename.drop("protocol_type", "service", "flag")

In [7]:
#Primeramente extraemos todas las columnas numericas excepto la columna label
numeric_only_col = df_only_numeric.columns[:37]

#Importamos VectorAsembler, es un transformador que combina una lista dada 
# de columnas en una sola columna vectorial
from pyspark.ml.feature import VectorAssembler
# Luego vectorizamos
df_va = VectorAssembler(inputCols = numeric_only_col, outputCol = "featureVector")
new_df = df_va.transform(df_only_numeric)


In [8]:
#Importamos el k-means para empezar a armar nuestro modelo
from pyspark.ml.clustering import KMeans

#Sin la especificación del valor de k en la inicialización de la clase KMeans, K toma por default el valor 2
kmeans = KMeans()
#Establecemos la columna de predicción
kmeans.setPredictionCol("cluster")
#Establecemos el feactureCol
kmeans.setFeaturesCol("featureVector")
#Creamos el modelo
model = kmeans.fit(new_df)

In [10]:
#Obtenemos los valores centros 
center = model.clusterCenters()
print(center)

[array([4.83401949e+01, 1.83462155e+03, 8.26203190e+02, 5.71611720e-06,
       6.48779303e-04, 7.96173468e-06, 1.24376586e-02, 3.20510858e-05,
       1.43529049e-01, 8.08830584e-03, 6.81851124e-05, 3.67464677e-05,
       1.29349608e-02, 1.18874823e-03, 7.43095237e-05, 1.02114351e-03,
       0.00000000e+00, 4.08294086e-07, 8.35165553e-04, 3.34973508e+02,
       2.95267146e+02, 1.77970317e-01, 1.78036989e-01, 5.76648988e-02,
       5.77299094e-02, 7.89884132e-01, 2.11796106e-02, 2.82608101e-02,
       2.32981078e+02, 1.89214283e+02, 7.53713390e-01, 3.07109788e-02,
       6.05051931e-01, 6.46410789e-03, 1.78091184e-01, 1.77885898e-01,
       5.79276115e-02]), array([1.0999000e+04, 0.0000000e+00, 1.3099374e+09, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,


In [14]:
#Como tenemos 23 distintos tipos de conexiones, podemos concluir que con k = 2 no es preciso nuestro modelo
# Por tanto no podemos precisar nuestras agrupaciones en nuestro datos
#Veremos como se grupo nuestro datos, volvemos a tranformar 
transformed = model.transform(new_df)

In [15]:
#Selecionamos la columna cluster y label
transformed.select("cluster","label").groupBy("cluster", "label").count().orderBy(["cluster","count"], ascending=[1,0]).show()

+-------+----------------+-------+
|cluster|           label|  count|
+-------+----------------+-------+
|      0|          smurf.|2807886|
|      0|        neptune.|1072017|
|      0|         normal.| 972781|
|      0|          satan.|  15892|
|      0|        ipsweep.|  12481|
|      0|      portsweep.|  10412|
|      0|           nmap.|   2316|
|      0|           back.|   2203|
|      0|    warezclient.|   1020|
|      0|       teardrop.|    979|
|      0|            pod.|    264|
|      0|   guess_passwd.|     53|
|      0|buffer_overflow.|     30|
|      0|           land.|     21|
|      0|    warezmaster.|     20|
|      0|           imap.|     12|
|      0|        rootkit.|     10|
|      0|     loadmodule.|      9|
|      0|      ftp_write.|      8|
|      0|       multihop.|      7|
|      0|            phf.|      4|
|      0|           perl.|      3|
|      0|            spy.|      2|
|      1|      portsweep.|      1|
+-------+----------------+-------+



In [ ]:
#Eligiendo el valor mas optimo o el mejor valor de k para nuestro conjunto de datos 
#Para cada cierto valor de K podriamos medir la calidad de la agrupación, es un buen parametro para saber que k 
# es el mas optimo, entonces para medir eso, una buena agrupación es cuandos los puntos estan mas cercano al centroide, que en este caso seria la media de las distancias o la media de las distancias al cuadrado
# Se define una funcion para ese calculo.
# Y K-means no ofrece un metodo computeCost para calcular esa media.
import random
def clusteringScore(data, k):
    vector = VectorAssembler(inputCols = numeric_only_col, outputCol = "featureVector")
    new_df = df_va.transform(data)
    kmeans = KMeans()
    kmeans.setSeed(random.randint(1,1000))
    kmeans.setK(k)
    kmeans.setPredictionCol("cluster")
    kmeans.setFeaturesCol("featureVector")
    model = kmeans.fit(new_df)
    return model.computeCost(new_df) / data.count()

#Comenzando por k igual 20 hasta 100 con paso de 20
for k in range(20,120,20):
    media = clusteringScore(df_only_numeric, k)
    print(k, media)

In [ ]:
#En base eso podemos ver que para ciertos valores de k en nuestro conjunto de datos tenemos que k=80 se tiene un agrupamiento suboptimo y ademas digamos que puede parar antes de llegar a su optimo local.
#Pero es importante siempre va a depender de nuestro valores iniciales de los centroides, normalmente se eligen de forma aleatoria, hay dos tipos de variantes que se encargan de eso que es k-means++ y K-means||, por default la libreia de spark implementa el k-means||
# Por tanto podemos ir ajustando algunos valores de nuestro modelo para tener un resultado mas optimo
# Como el numeros de iteraciones, y la cantidad minima de movimiento del centroide
# A valores menores el algortimo deja que los centroides se muevan por mucho mas tiempo
# Pero en ese caso ajustamos el numero maximo de iteracciones, para que no haga demasiado calculos
def clusteringScore(data, k):
    vector = VectorAssembler(inputCols = numeric_only_col, outputCol = "featureVector")
    new_df = df_va.transform(data)
    kmeans = KMeans()
    #Nuevo valores establecidos
    kmeans.setMaxIter(40)
    kmeans.setTol(1.0e-5)
    kmeans.setSeed(random.randint(1,1000))
    kmeans.setK(k)
    kmeans.setPredictionCol("cluster")
    kmeans.setFeaturesCol("featureVector")
    model = kmeans.fit(new_df)
    return model.computeCost(new_df) / data.count()

#Comenzando por k igual 20 hasta 100 con paso de 20
for k in range(20,120,20):
    media = clusteringScore(df_only_numeric, k)
    print(k, media)

In [ ]:
#Digamos que con estos ajuste se podria, queremos encontrar un punto a partir del cual el aumento de k no reduzca mucho esa media que obtenemos
#Por tanto podemos observa mediante los resultados que a partir de k igual a 100 disminuye esa media, podriamos encontrar un valor de k mas optimo por encima de los 100